In [94]:
import pandas as pd
import plotly.graph_objects as go
import streamlit as st
import plotly.express as px
from mainnet_launch.constants import (
    AutopoolConstants,
    STATS_CALCULATOR_REGISTRY,
    ALL_AUTOPOOLS,
    ETH_CHAIN,
    ROOT_PRICE_ORACLE,
    ChainData,
)
from mainnet_launch.data_fetching.get_events import fetch_events
from mainnet_launch.data_fetching.add_info_to_dataframes import add_timestamp_to_df_with_block_column
from mainnet_launch.abis import AUTOPOOL_VAULT_ABI


from mainnet_launch.database.database_operations import (
    write_dataframe_to_table,
    get_earliest_block_from_table_with_autopool,
    get_all_rows_in_table_by_autopool,
)
from mainnet_launch.database.should_update_database import should_update_table

import plotly.io as pio

pio.templates.default = None


from mainnet_launch.abis import STATS_CALCULATOR_REGISTRY_ABI


[e for e in stats_calculator_registry_contract.events]

[web3._utils.datatypes.FactorySet,
 web3._utils.datatypes.StatCalculatorRegistered,
 web3._utils.datatypes.StatCalculatorRemoved]

In [95]:
from mainnet_launch.data_fetching.add_info_to_dataframes import add_timestamp_to_df_with_block_column
from multicall import Call
from mainnet_launch.data_fetching.get_state_by_block import (
    identity_with_bool_success,
    get_raw_state_by_blocks,
    get_state_by_one_block,
    safe_normalize_with_bool_success,
    build_blocks_to_use,
)


def _fetch_lst_calc_addresses_df() -> pd.DataFrame:
    # 3 http calls total
    stats_calculator_registry_contract = ETH_CHAIN.client.eth.contract(
        STATS_CALCULATOR_REGISTRY(ETH_CHAIN), abi=STATS_CALCULATOR_REGISTRY_ABI
    )

    StatCalculatorRegistered = fetch_events(stats_calculator_registry_contract.events.StatCalculatorRegistered)

    lstTokenAddress_calls = [
        Call(
            a,
            ["lstTokenAddress()(address)"],
            [(a, identity_with_bool_success)],
        )
        for a in StatCalculatorRegistered["calculatorAddress"]
    ]

    calculator_to_lst_address = get_state_by_one_block(
        lstTokenAddress_calls, ETH_CHAIN.client.eth.block_number, chain=ETH_CHAIN
    )
    StatCalculatorRegistered["lst"] = StatCalculatorRegistered["calculatorAddress"].map(calculator_to_lst_address)
    lst_calcs = StatCalculatorRegistered[~StatCalculatorRegistered["lst"].isna()].copy()

    symbol_calls = [
        Call(
            a,
            ["symbol()(string)"],
            [(a, identity_with_bool_success)],
        )
        for a in lst_calcs["lst"]
    ]
    calculator_to_lst_address = get_state_by_one_block(symbol_calls, ETH_CHAIN.client.eth.block_number, chain=ETH_CHAIN)
    lst_calcs["symbol"] = lst_calcs["lst"].map(calculator_to_lst_address)

    token_symbols_to_ignore = ["OETH", "stETH", "eETH"]
    # skip stETH and eETH because they are captured in wstETH and weETH
    # skip OETH because we dropped it in October 2024,
    lst_calcs = lst_calcs[~lst_calcs["symbol"].isin(token_symbols_to_ignore)].copy()

    return lst_calcs[["lst", "symbol", "calculatorAddress"]]


def fetch_lst_stats_calculator_df(chain: ChainData, start_block: int) -> pd.DataFrame:
    if chain != ETH_CHAIN:
        raise ValueError("only for Ethereum")

    lst_calcs = _fetch_lst_calc_addresses_df()

    oracle_price_calls = [
        Call(
            ROOT_PRICE_ORACLE(ETH_CHAIN),
            ["getPriceInEth(address)(uint256)", lst],
            [(f"{symbol}_oracle_price", safe_normalize_with_bool_success)],
        )
        for (lst, symbol) in zip(lst_calcs["lst"], lst_calcs["symbol"])
    ]

    backing_calls = [
        Call(
            calculatorAddress,
            ["calculateEthPerToken()(uint256)"],
            [(f"{symbol}_backing", safe_normalize_with_bool_success)],
        )
        for (calculatorAddress, symbol) in zip(lst_calcs["calculatorAddress"], lst_calcs["symbol"])
    ]

    blocks = build_blocks_to_use(ETH_CHAIN, start_block=start_block)
    oracle_and_backing_df = get_raw_state_by_blocks([*oracle_price_calls, *backing_calls], blocks, ETH_CHAIN)
    return oracle_and_backing_df


def _raw_price_and_backing_data_to_long_format(wide_df: pd.DataFrame) -> pd.DataFrame:
    # takes the wide dataframe and makes it long instead
    # this is so that new LSTs can be added without breaking the table

    # timestamp	                    symbol	backing	    oracle_price
    # 0	2024-09-15 02:04:47+00:00	cbETH	1.081472	1.079332
    # 1	2024-09-15 08:06:23+00:00	cbETH	1.081472	1.079332

    backing_df = wide_df[[c for c in wide_df.columns if "_backing" in c]]
    long_backing_df = backing_df.reset_index().melt(id_vars=["timestamp"], var_name="symbol", value_name="backing")
    long_backing_df["symbol"] = long_backing_df["symbol"].apply(lambda x: str(x).replace("_backing", ""))

    oracle_price_df = wide_df[[c for c in wide_df.columns if "_oracle_price" in c]]
    long_oracle_price_df = oracle_price_df.reset_index().melt(
        id_vars=["timestamp"], var_name="symbol", value_name="oracle_price"
    )
    long_oracle_price_df["symbol"] = long_oracle_price_df["symbol"].apply(lambda x: str(x).replace("_oracle_price", ""))

    long_df = pd.merge(long_backing_df, long_oracle_price_df, on=["timestamp", "symbol"])
    return long_df


def _convert_long_df_to_wide_format(long_df: pd.DataFrame) -> pd.DataFrame:
    # convert the long_df (as it is stored on disk) and converts it back into the wide format
    # to use elsewhere
    # timestamp	cbETH_backing	cbETH_oracle_price
    # 2024-09-15 02:04:47+00:00	1.081472	1.079332
    # 2024-09-15 08:06:23+00:00	1.081472	1.079332

    # this does not maintain column order

    wide_backing_df = long_df[["timestamp", "symbol", "backing"]].pivot(
        index="timestamp", columns="symbol", values="backing"
    )
    wide_backing_df.columns = [f"{col}_backing" for col in wide_backing_df.columns]
    wide_backing_df.reset_index(inplace=True)

    wide_oracle_price_df = long_df[["timestamp", "symbol", "oracle_price"]].pivot(
        index="timestamp", columns="symbol", values="oracle_price"
    )
    wide_oracle_price_df.columns = [f"{col}_oracle_price" for col in wide_oracle_price_df.columns]
    wide_oracle_price_df.reset_index(inplace=True)

    wide_df = pd.merge(wide_backing_df, wide_oracle_price_df, on="timestamp")
    wide_df = wide_df.set_index("timestamp")
    return wide_df


oracle_and_backing_df = fetch_lst_stats_calculator_df(ETH_CHAIN, ETH_CHAIN.block_autopool_first_deployed)
oracle_and_backing_df

2025-01-21 18:06:57.432 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-21 18:06:57.433 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-21 18:06:57.433 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-21 18:06:57.435 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-21 18:06:57.435 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-21 18:06:58.399 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-21 18:06:58.400 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-21 18:06:58.400 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

,cbETH_oracle_price,ETHx_oracle_price,ezETH_oracle_price,frxETH_oracle_price,osETH_oracle_price,pxETH_oracle_price,rETH_oracle_price,rsETH_oracle_price,rswETH_oracle_price,swETH_oracle_price,...,ezETH_backing,frxETH_backing,osETH_backing,pxETH_backing,rETH_backing,rsETH_backing,rswETH_backing,swETH_backing,weETH_backing,wstETH_backing
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-15 02:04:47+00:00,1.079332,1.038034,1.017711,0.998047,1.028573,0.998678,1.117600,1.022689,1.016983,1.066387,...,1.019101,1.0,1.026487,1.0,1.115971,1.023909,1.018296,1.070436,1.047966,1.178811
2024-09-15 08:06:23+00:00,1.079332,1.038858,1.017699,0.998047,1.028634,0.998678,1.117600,1.022689,1.016983,1.066387,...,1.019105,1.0,1.026507,1.0,1.116040,1.024425,1.018296,1.070608,1.047966,1.178811
2024-09-15 14:07:35+00:00,1.079332,1.038858,1.017699,0.998047,1.028634,0.998678,1.117537,1.023337,1.016909,1.067016,...,1.019110,1.0,1.026527,1.0,1.116040,1.024425,1.018296,1.070608,1.047993,1.178902
2024-09-15 20:08:35+00:00,1.079332,1.038858,1.017727,0.998047,1.028634,1.001893,1.117537,1.023337,1.016909,1.067563,...,1.019113,1.0,1.026547,1.0,1.116040,1.024425,1.018296,1.070608,1.048020,1.178902
2024-09-16 02:09:35+00:00,1.079487,1.038858,1.017727,0.999145,1.028634,1.006186,1.117537,1.023337,1.016909,1.067458,...,1.019115,1.0,1.026566,1.0,1.116040,1.024425,1.018296,1.070608,1.048020,1.178902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-19 17:45:23+00:00,1.087274,1.048294,1.032422,0.996887,1.035035,0.996942,1.116270,1.032242,1.027326,1.078553,...,1.034036,1.0,1.036740,1.0,1.126432,1.034637,1.031021,1.081717,1.057992,1.191132
2025-01-19 23:47:11+00:00,1.087209,1.048294,1.032240,0.993565,1.035035,0.996920,1.116270,1.032242,1.027326,1.078496,...,1.034038,1.0,1.036761,1.0,1.126432,1.034637,1.031021,1.081717,1.058005,1.191132
2025-01-20 05:48:11+00:00,1.087209,1.048049,1.032240,1.002354,1.035035,0.996898,1.116270,1.032242,1.025014,1.078410,...,1.033867,1.0,1.036783,1.0,1.126432,1.034706,1.031021,1.081717,1.058032,1.191132


In [117]:
a_long_df = _raw_price_and_backing_data_to_long_format(oracle_and_backing_df)
a_long_df

,timestamp,symbol,backing,oracle_price
0,2024-09-15 02:04:47+00:00,cbETH,1.081472,1.079332
1,2024-09-15 08:06:23+00:00,cbETH,1.081472,1.079332
2,2024-09-15 14:07:35+00:00,cbETH,1.081472,1.079332
3,2024-09-15 20:08:35+00:00,cbETH,1.081546,1.079332
4,2024-09-16 02:09:35+00:00,cbETH,1.081546,1.079487
...,...,...,...,...
6103,2025-01-19 17:45:23+00:00,wstETH,1.191132,1.189097
6104,2025-01-19 23:47:11+00:00,wstETH,1.191132,1.189097
6105,2025-01-20 05:48:11+00:00,wstETH,1.191132,1.188326
6106,2025-01-20 11:49:23+00:00,wstETH,1.191132,1.188326


In [116]:
oracle_and_backing_df.dtypes

cbETH_oracle_price     float64
ETHx_oracle_price      float64
ezETH_oracle_price     float64
frxETH_oracle_price    float64
osETH_oracle_price     float64
pxETH_oracle_price     float64
rETH_oracle_price      float64
rsETH_oracle_price     float64
rswETH_oracle_price    float64
swETH_oracle_price     float64
weETH_oracle_price     float64
wstETH_oracle_price    float64
cbETH_backing          float64
ETHx_backing           float64
ezETH_backing          float64
frxETH_backing         float64
osETH_backing          float64
pxETH_backing          float64
rETH_backing           float64
rsETH_backing          float64
rswETH_backing         float64
swETH_backing          float64
weETH_backing          float64
wstETH_backing         float64
dtype: object

In [118]:
a_wide_df = _convert_long_df_to_wide_format(a_long_df)
a_wide_df.dtypes

ETHx_backing           float64
cbETH_backing          float64
ezETH_backing          float64
frxETH_backing         float64
osETH_backing          float64
pxETH_backing          float64
rETH_backing           float64
rsETH_backing          float64
rswETH_backing         float64
swETH_backing          float64
weETH_backing          float64
wstETH_backing         float64
ETHx_oracle_price      float64
cbETH_oracle_price     float64
ezETH_oracle_price     float64
frxETH_oracle_price    float64
osETH_oracle_price     float64
pxETH_oracle_price     float64
rETH_oracle_price      float64
rsETH_oracle_price     float64
rswETH_oracle_price    float64
swETH_oracle_price     float64
weETH_oracle_price     float64
wstETH_oracle_price    float64
dtype: object

In [119]:
for col in a_wide_df.columns:
    if not all(a_wide_df[col].values == oracle_and_backing_df[col].values):
        print(col)

rsETH_oracle_price


In [137]:
oracle_and_backing_df[oracle_and_backing_df.isna().any(axis=1)]

,cbETH_oracle_price,ETHx_oracle_price,ezETH_oracle_price,frxETH_oracle_price,osETH_oracle_price,pxETH_oracle_price,rETH_oracle_price,rsETH_oracle_price,rswETH_oracle_price,swETH_oracle_price,...,ezETH_backing,frxETH_backing,osETH_backing,pxETH_backing,rETH_backing,rsETH_backing,rswETH_backing,swETH_backing,weETH_backing,wstETH_backing
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-11-24 10:08:23+00:00,1.08255,1.044832,1.025613,0.986059,1.032401,0.997801,1.112429,NaN,1.021001,1.070916,...,1.027827,1.0,1.032317,1.0,1.121761,1.030007,1.024781,1.076767,1.053612,1.185716


In [141]:
a_wide_df[a_wide_df.isna().any(axis=1)]["rsETH_oracle_price"] == oracle_and_backing_df[
    oracle_and_backing_df.isna().any(axis=1)
]["rsETH_oracle_price"]

timestamp
2024-11-24 10:08:23+00:00    False
Name: rsETH_oracle_price, dtype: bool

In [142]:
a_wide_df[a_wide_df.isna().any(axis=1)]["rsETH_oracle_price"], oracle_and_backing_df[
    oracle_and_backing_df.isna().any(axis=1)
]["rsETH_oracle_price"]

(timestamp
 2024-11-24 10:08:23+00:00   NaN
 Name: rsETH_oracle_price, dtype: float64,
 timestamp
 2024-11-24 10:08:23+00:00   NaN
 Name: rsETH_oracle_price, dtype: float64)

In [135]:
a_wide_df.isna().sum()

ETHx_backing           0
cbETH_backing          0
ezETH_backing          0
frxETH_backing         0
osETH_backing          0
pxETH_backing          0
rETH_backing           0
rsETH_backing          0
rswETH_backing         0
swETH_backing          0
weETH_backing          0
wstETH_backing         0
ETHx_oracle_price      0
cbETH_oracle_price     0
ezETH_oracle_price     0
frxETH_oracle_price    0
osETH_oracle_price     0
pxETH_oracle_price     0
rETH_oracle_price      0
rsETH_oracle_price     1
rswETH_oracle_price    0
swETH_oracle_price     0
weETH_oracle_price     0
wstETH_oracle_price    0
dtype: int64

In [130]:
df = oracle_and_backing_df[["rsETH_oracle_price"]].copy()
df["copy"] = a_wide_df["rsETH_oracle_price"].values
diff = df["copy"] - df["rsETH_oracle_price"]

df.isna().sum()
# px.line(diff)

rsETH_oracle_price    1
copy                  1
dtype: int64

In [131]:
px.line(df)

In [121]:
px.line(a_wide_df["rsETH_oracle_price"], oracle_and_backing_df["rsETH_oracle_price"])

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [108]:
backing_df = oracle_and_backing_df[[c for c in oracle_and_backing_df.columns if "_backing" in c]]

long_backing_df = backing_df.reset_index().melt(id_vars=["timestamp"], var_name="symbol", value_name="backing")
long_backing_df["symbol"] = long_backing_df["symbol"].apply(lambda x: str(x).replace("_backing", ""))


oracle_price_df = oracle_and_backing_df[[c for c in oracle_and_backing_df.columns if "_oracle_price" in c]]
long_oracle_price_df = oracle_price_df.reset_index().melt(
    id_vars=["timestamp"], var_name="symbol", value_name="oracle_price"
)
long_oracle_price_df["symbol"] = long_oracle_price_df["symbol"].apply(lambda x: str(x).replace("_oracle_price", ""))

long_df = pd.merge(long_backing_df, long_oracle_price_df, on=["timestamp", "symbol"])
long_df

,timestamp,symbol,backing,oracle_price
0,2024-09-15 02:04:47+00:00,cbETH,1.081472,1.079332
1,2024-09-15 08:06:23+00:00,cbETH,1.081472,1.079332
2,2024-09-15 14:07:35+00:00,cbETH,1.081472,1.079332
3,2024-09-15 20:08:35+00:00,cbETH,1.081546,1.079332
4,2024-09-16 02:09:35+00:00,cbETH,1.081546,1.079487
...,...,...,...,...
6103,2025-01-19 17:45:23+00:00,wstETH,1.191132,1.189097
6104,2025-01-19 23:47:11+00:00,wstETH,1.191132,1.189097
6105,2025-01-20 05:48:11+00:00,wstETH,1.191132,1.188326
6106,2025-01-20 11:49:23+00:00,wstETH,1.191132,1.188326


In [105]:
StatCalculatorRegistered = add_timestamp_to_df_with_block_column(
    fetch_events(stats_calculator_registry_contract.events.StatCalculatorRegistered), chain=ETH_CHAIN
)
StatCalculatorRegistered.head()

2025-01-21 18:10:01.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-21 18:10:01.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-21 18:10:01.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-21 18:10:01.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-21 18:10:01.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


,aprId,calculatorAddress,caller,event,block,transaction_index,log_index,hash
timestamp,,,,,,,,
2024-08-31 12:50:11+00:00,b'\x9f\xd8\xa3i\x9a)lz&\xa7-N-\xfb\x01\x9d@\x9...,0xaB78a570252dd06FDbC1C5C566E842e571d01E08,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648662,126,269,0xc753bec65f46636fde7763c511c7954e493020515a2e...
2024-08-31 12:50:35+00:00,b'\x99\x14\xfb>\x17#\xdfL\xab2\xa7\xfd\xa8s\xf...,0x4353e181C13f7E970f24016a0762C1af271350BA,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648664,161,384,0x8dc5722219bba6c03951bdfb4a6e4f4c77a7116fea85...
2024-08-31 12:50:47+00:00,b'\x0f\x82O-\xd5_\x04+\xb8\x8a\xed\xf4[\x03\xc...,0x6D3C5F6670ABe46901De4BD39036cF21d178334C,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648665,140,277,0x15f1ea43aaff02ef140b92ee5359e990661779295e2f...
2024-08-31 12:50:59+00:00,"b':nPf,\t\xfe\x07\x0c\xfa+\x8b\xbbY\x7fT\xdeE\...",0xA84cc1d5aD1cDD5fAeb15Aa3f4aC5935D4b263D9,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648666,111,225,0x2323ec23cb9feb1b65c18cec93d989662a8b91a8d5b8...
2024-08-31 12:51:11+00:00,"b'\x0be\xef\xf3\xe2\x1e#\x18[F\xbb\xa8,\xefc%S...",0x449A957490e24e4d915fD5Dcf25Dd5446E787590,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648667,143,758,0x0cb8aeee88852e110136b2ec4e950676e7bc87f8c256...


In [106]:
from mainnet_launch.data_fetching.add_info_to_dataframes import add_timestamp_to_df_with_block_column
from multicall import Call
from mainnet_launch.data_fetching.get_state_by_block import (
    identity_with_bool_success,
    get_raw_state_by_blocks,
    get_state_by_one_block,
    safe_normalize_with_bool_success,
    build_blocks_to_use,
)

lstTokenAddress_calls = [
    Call(
        a,
        ["lstTokenAddress()(address)"],
        [(a, identity_with_bool_success)],
    )
    for a in StatCalculatorRegistered["calculatorAddress"]
]

calculator_to_lst_address = get_state_by_one_block(
    lstTokenAddress_calls, ETH_CHAIN.client.eth.block_number, chain=ETH_CHAIN
)
StatCalculatorRegistered["lst"] = StatCalculatorRegistered["calculatorAddress"].map(calculator_to_lst_address)
lst_calcs = StatCalculatorRegistered[~StatCalculatorRegistered["lst"].isna()].copy()
lst_calcs

,aprId,calculatorAddress,caller,event,block,transaction_index,log_index,hash,lst
timestamp,,,,,,,,,
2024-08-31 12:50:11+00:00,b'\x9f\xd8\xa3i\x9a)lz&\xa7-N-\xfb\x01\x9d@\x9...,0xaB78a570252dd06FDbC1C5C566E842e571d01E08,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648662,126,269,0xc753bec65f46636fde7763c511c7954e493020515a2e...,0xbe9895146f7af43049ca1c1ae358b0541ea49704
2024-08-31 12:50:35+00:00,b'\x99\x14\xfb>\x17#\xdfL\xab2\xa7\xfd\xa8s\xf...,0x4353e181C13f7E970f24016a0762C1af271350BA,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648664,161,384,0x8dc5722219bba6c03951bdfb4a6e4f4c77a7116fea85...,0x35fa164735182de50811e8e2e824cfb9b6118ac2
2024-08-31 12:50:47+00:00,b'\x0f\x82O-\xd5_\x04+\xb8\x8a\xed\xf4[\x03\xc...,0x6D3C5F6670ABe46901De4BD39036cF21d178334C,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648665,140,277,0x15f1ea43aaff02ef140b92ee5359e990661779295e2f...,0xa35b1b31ce002fbf2058d22f30f95d405200a15b
2024-08-31 12:50:59+00:00,"b':nPf,\t\xfe\x07\x0c\xfa+\x8b\xbbY\x7fT\xdeE\...",0xA84cc1d5aD1cDD5fAeb15Aa3f4aC5935D4b263D9,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648666,111,225,0x2323ec23cb9feb1b65c18cec93d989662a8b91a8d5b8...,0xbf5495efe5db9ce00f80364c8b423567e58d2110
2024-08-31 12:51:11+00:00,"b'\x0be\xef\xf3\xe2\x1e#\x18[F\xbb\xa8,\xefc%S...",0x449A957490e24e4d915fD5Dcf25Dd5446E787590,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648667,143,758,0x0cb8aeee88852e110136b2ec4e950676e7bc87f8c256...,0x5e8422345238f34275888049021821e8e08caa1f
2024-08-31 12:51:23+00:00,b'_\xb0u\xfc_\xbbgOfHkM\x12?l\no\x1c\xda\xa5\x...,0xB5aa595C4FE3C297D65bdCDCc6FA48eF8725AEbB,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648668,146,367,0x982f6215438b2a538b51016acce61d4da7cefaf71b19...,0xf1c9acdc66974dfb6decb12aa385b9cd01190e38
2024-08-31 12:51:35+00:00,b'L/_\xf4\xcc<p\xc8\xa7l\x07\xce@\\\xf6\xdcY\x...,0x9cB562083D29e027F21fAc4D8b66573deA972153,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648669,131,378,0x279e10156a63ada938fba288c438642dfd4f5fedd005...,0x04c154b66cb340f3ae24111cc767e0184ed00cc6
2024-08-31 12:51:47+00:00,b'\xb5\x07\xdc\x9b\xf7B\x0e\xf8\xee\xd4\x1f~3>...,0x9801098EE481ed6806C61A4dE259FBdDD5bb84a8,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648670,86,214,0x6f13657889047704a3c03ebfafd2c0926bd7bab88536...,0xae78736cd615f374d3085123a210448e74fc6393
2024-08-31 12:51:59+00:00,b'\xcd%\xe0\x8c\x9c\x95\x05\xb4\x01\xeel_\x1a\...,0x840A49a4b83E57718cf67c03D820C938A04FC210,0x8198069F1709aDDC6A162c9433665776E5Bc917c,StatCalculatorRegistered,20648671,139,820,0x157ad839030ce48754131b5b1bf8cc114e1c6702fbda...,0xa1290d69c65a6fe4df752f95823fae25cb99e5a7


In [99]:
backing_df = 


SyntaxError: invalid syntax (2178141823.py, line 1)

In [73]:
cols = []
for symbol in lst_calcs["symbol"].values:
    # only look at the wrapped version of the token
    if symbol in ["OETH"]:
        # 0ETH stopped being supported on Oct 24, 2024, excluding it
        continue

    if symbol not in ["stETH", "eETH"]:
        oracle_and_backing_df[f"{symbol}_discount"] = 100 - (
            100 * oracle_and_backing_df[f"{symbol}_oracle"] / oracle_and_backing_df[f"{symbol}_backing"]
        )
        cols.append(f"{symbol}_discount")

In [78]:
px.line(oracle_and_backing_df)

In [83]:
px.line(oracle_and_backing_df[cols], title="LST / LRT Percent Discount")

In [75]:
lst_calcs[["symbol", "lst", "calculatorAddress"]]

,symbol,lst,calculatorAddress
timestamp,,,
2024-08-31 12:50:11+00:00,cbETH,0xbe9895146f7af43049ca1c1ae358b0541ea49704,0xaB78a570252dd06FDbC1C5C566E842e571d01E08
2024-08-31 12:50:35+00:00,eETH,0x35fa164735182de50811e8e2e824cfb9b6118ac2,0x4353e181C13f7E970f24016a0762C1af271350BA
2024-08-31 12:50:47+00:00,ETHx,0xa35b1b31ce002fbf2058d22f30f95d405200a15b,0x6D3C5F6670ABe46901De4BD39036cF21d178334C
2024-08-31 12:50:59+00:00,ezETH,0xbf5495efe5db9ce00f80364c8b423567e58d2110,0xA84cc1d5aD1cDD5fAeb15Aa3f4aC5935D4b263D9
2024-08-31 12:51:11+00:00,frxETH,0x5e8422345238f34275888049021821e8e08caa1f,0x449A957490e24e4d915fD5Dcf25Dd5446E787590
2024-08-31 12:51:23+00:00,osETH,0xf1c9acdc66974dfb6decb12aa385b9cd01190e38,0xB5aa595C4FE3C297D65bdCDCc6FA48eF8725AEbB
2024-08-31 12:51:35+00:00,pxETH,0x04c154b66cb340f3ae24111cc767e0184ed00cc6,0x9cB562083D29e027F21fAc4D8b66573deA972153
2024-08-31 12:51:47+00:00,rETH,0xae78736cd615f374d3085123a210448e74fc6393,0x9801098EE481ed6806C61A4dE259FBdDD5bb84a8
2024-08-31 12:51:59+00:00,rsETH,0xa1290d69c65a6fe4df752f95823fae25cb99e5a7,0x840A49a4b83E57718cf67c03D820C938A04FC210


In [76]:
# we are not supporting 0ETH anymore, last napshot was Oct 24, 2024